# Benchmark of various outlier detection models

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). The full result by averaging 10 indepent trials can be found [here](https://pyod.readthedocs.io/en/latest/benchmark.html).

**[PyOD](https://github.com/yzhao062/pyod)** is a comprehensive **Python toolkit** to **identify outlying objects** in 
multivariate data with both unsupervised and supervised approaches.
The model covered in this example includes:

  1. Linear Models for Outlier Detection:
     1. **PCA: Principal Component Analysis** use the sum of
       weighted projected distances to the eigenvector hyperplane 
       as the outlier outlier scores)
     2. **MCD: Minimum Covariance Determinant** (use the mahalanobis distances 
       as the outlier scores)
     3. **OCSVM: One-Class Support Vector Machines**
     
  2. Proximity-Based Outlier Detection Models:
     1. **LOF: Local Outlier Factor**
     2. **CBLOF: Clustering-Based Local Outlier Factor**
     3. **kNN: k Nearest Neighbors** (use the distance to the kth nearest 
     neighbor as the outlier score)
     4. **HBOS: Histogram-based Outlier Score**
     
  3. Probabilistic Models for Outlier Detection:
     1. **ABOD: Angle-Based Outlier Detection**
  
  4. Outlier Ensembles and Combination Frameworks
     1. **Isolation Forest**
     2. **Feature Bagging**

     
Corresponding file could be found at /examples/compare_all_models.py

In [8]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
import tsfresh as tsf

In [5]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    print(X.shape)
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

#         roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
#         prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} execution time: {duration}s'.format(
            clf_name=clf_name, duration=duration))

#         roc_list.append(roc)
#         prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

#     temp_df = pd.DataFrame(roc_list).transpose()
#     temp_df.columns = df_columns
#     roc_df = pd.concat([roc_df, temp_df], axis=0)

#     temp_df = pd.DataFrame(prn_list).transpose()
#     temp_df.columns = df_columns
#     prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
(452, 274)
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.1631s
Cluster-based Local Outlier Factor ROC:0.7789, precision @ rank n:0.4643, execution time: 0.1755s
Feature Bagging ROC:0.7796, precision @ rank n:0.4643, execution time: 0.6477s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.0625s
Isolation Forest ROC:0.8639, precision @ rank n:0.6071, execution time: 0.3484s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.1047s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0801s
Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 0.7039s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.053s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0637s

... Processing cardio.mat ...
(1831, 21)
An

Minimum Covariance Determinant (MCD) ROC:0.3822, precision @ rank n:0.0, execution time: 1.2872s
One-class SVM (OCSVM) ROC:0.5171, precision @ rank n:0.0, execution time: 1.5614s
Principal Component Analysis (PCA) ROC:0.526, precision @ rank n:0.0, execution time: 0.0468s

... Processing pendigits.mat ...
(6870, 16)
Angle-based Outlier Detector (ABOD) ROC:0.667, precision @ rank n:0.0526, execution time: 1.7084s
Cluster-based Local Outlier Factor ROC:0.8082, precision @ rank n:0.1579, execution time: 0.2808s
Feature Bagging ROC:0.4889, precision @ rank n:0.0526, execution time: 4.2419s
Histogram-base Outlier Detection (HBOS) ROC:0.9348, precision @ rank n:0.2632, execution time: 0.0156s
Isolation Forest ROC:0.9376, precision @ rank n:0.3333, execution time: 0.8036s
K Nearest Neighbors (KNN) ROC:0.7371, precision @ rank n:0.0702, execution time: 0.7971s
Local Outlier Factor (LOF) ROC:0.4965, precision @ rank n:0.0702, execution time: 0.7181s
Minimum Covariance Determinant (MCD) ROC:0.82

In [12]:
data = pd.read_csv('../Data/123.csv')
data = data[['dateTime','iActual2', 'pha2']]
length = data.shape[0]
data['id'] = range(0,length)

In [14]:
feature_extracted = tsf.extract_features(data,column_id='id',column_sort='dateTime')
need_list = ['abs_energy','absolute_sum_of_changes','agg_autocorrelation__f_agg_"mean"__maxlag_40',
             'agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"','approximate_entropy__m_2__r_0.1',
             'ar_coefficient__k_10__coeff_0','augmented_dickey_fuller__autolag_"AIC"__attr_"pvalue"','binned_entropy__max_bins_10',
             'count_above_mean','count_below_mean','cid_ce__normalize_False','cid_ce__normalize_True',
             'cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_2','fft_aggregated__aggtype_"skew"',
             'fft_coefficient__coeff_2__attr_"angle"','linear_trend__attr_"pvalue"','number_crossing_m__m_1']

columns_all = list(feature_extracted.columns)
columns_need = [x0 for x0 in columns_all for y in need_list if y in x0]

feature_extracted = feature_extracted[columns_need]

Feature Extraction: 100%|█████████████████████████| 10/10 [00:21<00:00,  1.72s/it]


In [16]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train_norm)
    test_scores = clf.decision_function(X_test_norm)
    t1 = time()
    duration = round(t1 - t0, ndigits=4)
    time_list.append(duration)

#         roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
#         prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

    print('{clf_name} execution time: {duration}s'.format(
            clf_name=clf_name, duration=duration))

#         roc_list.append(roc)
#         prn_list.append(prn)

Angle-based Outlier Detector (ABOD) execution time: 0.0948s
Cluster-based Local Outlier Factor execution time: 0.0728s
Feature Bagging execution time: 0.0878s
Histogram-base Outlier Detection (HBOS) execution time: 0.0119s
Isolation Forest execution time: 0.2314s
K Nearest Neighbors (KNN) execution time: 0.0219s
Local Outlier Factor (LOF) execution time: 0.014s
Minimum Covariance Determinant (MCD) execution time: 0.0868s
One-class SVM (OCSVM) execution time: 0.009s
Principal Component Analysis (PCA) execution time: 0.005s


In [ ]:
print('Time complexity')
time_df

Analyze the performance of ROC and Precision @ n